In [14]:
import polars as pl
import os

In [19]:
volunteers = pl.scan_parquet("dataset/gz_decals_volunteers_1_and_2.parquet")
volunteers.head().collect()

iauname,ra,dec,redshift,elpetro_absmag_r,sersic_nmgy_r,petro_th50,petro_th90,petro_theta,upload_group,active_learning_on,in_gzd_c,data_release,smooth-or-featured_total-votes,smooth-or-featured_smooth,smooth-or-featured_smooth_fraction,smooth-or-featured_smooth_debiased,smooth-or-featured_featured-or-disk,smooth-or-featured_featured-or-disk_fraction,smooth-or-featured_featured-or-disk_debiased,smooth-or-featured_artifact,smooth-or-featured_artifact_fraction,smooth-or-featured_artifact_debiased,how-rounded_total-votes,how-rounded_completely,how-rounded_completely_fraction,how-rounded_completely_debiased,how-rounded_in-between,how-rounded_in-between_fraction,how-rounded_in-between_debiased,how-rounded_cigar-shaped,how-rounded_cigar-shaped_fraction,how-rounded_cigar-shaped_debiased,disk-edge-on_total-votes,disk-edge-on_yes,disk-edge-on_yes_fraction,disk-edge-on_yes_debiased,…,spiral-arm-count_2,spiral-arm-count_2_fraction,spiral-arm-count_2_debiased,spiral-arm-count_3,spiral-arm-count_3_fraction,spiral-arm-count_3_debiased,spiral-arm-count_4,spiral-arm-count_4_fraction,spiral-arm-count_4_debiased,spiral-arm-count_more-than-4,spiral-arm-count_more-than-4_fraction,spiral-arm-count_more-than-4_debiased,bulge-size_total-votes,bulge-size_none,bulge-size_none_fraction,bulge-size_none_debiased,bulge-size_obvious,bulge-size_obvious_fraction,bulge-size_obvious_debiased,bulge-size_dominant,bulge-size_dominant_fraction,bulge-size_dominant_debiased,merging_total-votes,merging_merger,merging_merger_fraction,merging_merger_debiased,merging_tidal-debris,merging_tidal-debris_fraction,merging_tidal-debris_debiased,merging_both,merging_both_fraction,merging_both_debiased,merging_neither,merging_neither_fraction,merging_neither_debiased,wrong_size_statistic,wrong_size_warning
str,f64,f64,f32,f32,f32,f32,f32,f32,str,bool,bool,str,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,…,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,f64,bool
"""J103438.28-005109.6""",158.659544,-0.85268,0.073783,-21.083954,520.810913,2.88914,6.546928,5.985117,"""pre_active""",false,true,"""DR1""",39,19,0.487179,0.115971,15,0.384615,0.804878,5,0.128205,0.157894,19,16,0.842105,0.870968,2,0.105263,0.12,1,0.052632,0.133333,15,2,0.133333,0.113636,…,5,0.625,0.265582,1,0.125,0.294883,1,0.125,0.343777,1,0.125,0.382353,13,0,0.0,0.0,12,0.923077,0.904762,1,0.076923,0.095238,34,5,0.147059,0.143352,3,0.088235,0.088889,3,0.088235,0.189189,23,0.676471,0.742857,152.781368,false
"""J103542.82-004509.8""",158.928432,-0.752741,0.115605,-20.905863,157.557159,1.599772,4.8975,3.493677,"""pre_active""",false,true,"""DR1""",36,31,0.861111,0.386868,2,0.055556,0.16129,3,0.083333,0.096525,31,27,0.870968,0.870968,4,0.129032,0.129032,0,0.0,0.0,2,1,0.5,0.16,…,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,1,0,0.0,0.0,1,1.0,1.0,0,0.0,0.0,33,6,0.181818,0.173006,3,0.090909,0.130435,1,0.030303,0.033333,23,0.69697,0.680851,152.214574,false
"""J103501.69-004952.4""",158.757099,-0.831234,0.118851,-19.841774,58.042347,1.326934,3.000635,2.698255,"""pre_active""",false,false,"""DR1""",41,33,0.804878,0.283874,1,0.02439,0.592593,7,0.170732,0.217691,33,21,0.636364,0.372093,12,0.363636,0.487179,0,0.0,0.0,1,0,0.0,0.0,…,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,1,0,0.0,0.0,1,1.0,1.0,0,0.0,0.0,34,1,0.029412,0.033961,1,0.029412,0.1,0,0.0,0.0,32,0.941176,0.869565,160.183576,false
"""J103649.87-003753.2""",159.20782,-0.63147,0.06356,-19.598785,173.327042,3.622341,7.018666,6.951218,"""pre_active""",false,true,"""DR1""",49,31,0.632653,0.275341,14,0.285714,0.864865,4,0.081633,0.096841,31,15,0.483871,0.483871,16,0.516129,0.516129,0,0.0,0.0,14,0,0.0,0.0,…,4,0.571429,0.243534,0,0.0,0.0,2,0.285714,0.522109,1,0.142857,0.304348,14,4,0.285714,0.15241,10,0.714286,0.764706,0,0.0,0.0,45,1,0.022222,0.025072,2,0.044444,0.054545,1,0.022222,0.033333,41,0.911111,0.885714,158.02642,false
"""J103647.21-004056.4"""

In [20]:
volunteers.filter(pl.col("data_release") == "DR5").head().collect()

iauname,ra,dec,redshift,elpetro_absmag_r,sersic_nmgy_r,petro_th50,petro_th90,petro_theta,upload_group,active_learning_on,in_gzd_c,data_release,smooth-or-featured_total-votes,smooth-or-featured_smooth,smooth-or-featured_smooth_fraction,smooth-or-featured_smooth_debiased,smooth-or-featured_featured-or-disk,smooth-or-featured_featured-or-disk_fraction,smooth-or-featured_featured-or-disk_debiased,smooth-or-featured_artifact,smooth-or-featured_artifact_fraction,smooth-or-featured_artifact_debiased,how-rounded_total-votes,how-rounded_completely,how-rounded_completely_fraction,how-rounded_completely_debiased,how-rounded_in-between,how-rounded_in-between_fraction,how-rounded_in-between_debiased,how-rounded_cigar-shaped,how-rounded_cigar-shaped_fraction,how-rounded_cigar-shaped_debiased,disk-edge-on_total-votes,disk-edge-on_yes,disk-edge-on_yes_fraction,disk-edge-on_yes_debiased,…,spiral-arm-count_2,spiral-arm-count_2_fraction,spiral-arm-count_2_debiased,spiral-arm-count_3,spiral-arm-count_3_fraction,spiral-arm-count_3_debiased,spiral-arm-count_4,spiral-arm-count_4_fraction,spiral-arm-count_4_debiased,spiral-arm-count_more-than-4,spiral-arm-count_more-than-4_fraction,spiral-arm-count_more-than-4_debiased,bulge-size_total-votes,bulge-size_none,bulge-size_none_fraction,bulge-size_none_debiased,bulge-size_obvious,bulge-size_obvious_fraction,bulge-size_obvious_debiased,bulge-size_dominant,bulge-size_dominant_fraction,bulge-size_dominant_debiased,merging_total-votes,merging_merger,merging_merger_fraction,merging_merger_debiased,merging_tidal-debris,merging_tidal-debris_fraction,merging_tidal-debris_debiased,merging_both,merging_both_fraction,merging_both_debiased,merging_neither,merging_neither_fraction,merging_neither_debiased,wrong_size_statistic,wrong_size_warning
str,f64,f64,f32,f32,f32,f32,f32,f32,str,bool,bool,str,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,…,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,f64,bool


In [ ]:
def add_image_path(df: pl.LazyFrame, dataset_dir: str = "./dataset") -> pl.LazyFrame:
    # check if file exists
    def file_exists(path: str) -> bool:
        return os.path.exists(os.path.join(dataset_dir, path))

    df = df.with_columns(
        (pl.col("iauname").str.slice(0, 4) + "/" + pl.col("iauname") + ".png").alias(
            "image_path"
        ),
    ).filter(pl.col("image_path").map_elements(file_exists, return_dtype=pl.Boolean))

    # reorder columns to have 'image_path' first
    df = df.select(
        pl.col("image_path"),
        *df.drop("image_path").collect_schema().names()
    )

    return df


volunteers = add_image_path(volunteers)
volunteers.head().collect()

/tmp/ipykernel_10263/3019175325.py:15: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  *df.drop("image_path").columns


image_path,iauname,ra,dec,redshift,elpetro_absmag_r,sersic_nmgy_r,petro_th50,petro_th90,petro_theta,upload_group,active_learning_on,in_gzd_c,data_release,smooth-or-featured_total-votes,smooth-or-featured_smooth,smooth-or-featured_smooth_fraction,smooth-or-featured_smooth_debiased,smooth-or-featured_featured-or-disk,smooth-or-featured_featured-or-disk_fraction,smooth-or-featured_featured-or-disk_debiased,smooth-or-featured_artifact,smooth-or-featured_artifact_fraction,smooth-or-featured_artifact_debiased,how-rounded_total-votes,how-rounded_completely,how-rounded_completely_fraction,how-rounded_completely_debiased,how-rounded_in-between,how-rounded_in-between_fraction,how-rounded_in-between_debiased,how-rounded_cigar-shaped,how-rounded_cigar-shaped_fraction,how-rounded_cigar-shaped_debiased,disk-edge-on_total-votes,disk-edge-on_yes,disk-edge-on_yes_fraction,…,spiral-arm-count_2,spiral-arm-count_2_fraction,spiral-arm-count_2_debiased,spiral-arm-count_3,spiral-arm-count_3_fraction,spiral-arm-count_3_debiased,spiral-arm-count_4,spiral-arm-count_4_fraction,spiral-arm-count_4_debiased,spiral-arm-count_more-than-4,spiral-arm-count_more-than-4_fraction,spiral-arm-count_more-than-4_debiased,bulge-size_total-votes,bulge-size_none,bulge-size_none_fraction,bulge-size_none_debiased,bulge-size_obvious,bulge-size_obvious_fraction,bulge-size_obvious_debiased,bulge-size_dominant,bulge-size_dominant_fraction,bulge-size_dominant_debiased,merging_total-votes,merging_merger,merging_merger_fraction,merging_merger_debiased,merging_tidal-debris,merging_tidal-debris_fraction,merging_tidal-debris_debiased,merging_both,merging_both_fraction,merging_both_debiased,merging_neither,merging_neither_fraction,merging_neither_debiased,wrong_size_statistic,wrong_size_warning
str,str,f64,f64,f32,f32,f32,f32,f32,f32,str,bool,bool,str,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,…,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,f64,f64,bool
"""J000/J000309.06-004508.1.png""","""J000309.06-004508.1""",0.787756,-0.752299,0.029348,-17.539028,123.711609,2.724064,7.544131,5.538606,"""pre_active""",false,true,"""DR1""",39,29,0.74359,0.656996,7,0.179487,0.210526,3,0.076923,0.082545,29,4,0.137931,0.137931,24,0.827586,0.783784,1,0.034483,0.045455,7,2,0.285714,…,4,1.0,1.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,5,3,0.6,0.573606,1,0.2,0.107143,1,0.2,0.157895,36,0,0.0,0.0,5,0.138889,0.21875,2,0.055556,0.064516,29,0.805556,0.711538,153.626286,false
"""J000/J000256.55-004733.7.png""","""J000256.55-004733.7""",0.735658,-0.79273,0.12686,-21.441341,272.491425,3.62516,9.373409,8.043182,"""pre_active""",false,true,"""DR1""",49,10,0.204082,0.033096,36,0.734694,0.935484,3,0.061224,0.06743,10,5,0.5,0.423077,5,0.5,0.566667,0,0.0,0.0,36,1,0.027778,…,12,0.75,0.399694,1,0.0625,0.168597,0,0.0,0.0,2,0.125,0.36,35,0,0.0,0.0,24,0.685714,0.68,11,0.314286,0.321429,46,3,0.065217,0.068971,6,0.130435,0.159091,2,0.043478,0.066667,35,0.76087,0.767442,157.019848,false
"""J000/J000358.11-000244.3.png""","""J000358.11-000244.3""",0.992144,-0.04564,0.090954,-20.715004,226.361069,4.548683,11.156693,9.522999,"""pre_active""",false,true,"""DR1""",54,9,0.166667,0.040515,42,0.777778,0.914894,3,0.055556,0.064092,9,6,0.666667,0.566667,3,0.333333,0.368421,0,0.0,0.0,42,1,0.02381,…,18,0.486486,0.207893,7,0.189189,0.275258,2,0.054054,0.164479,0,0.0,0.0,41,2,0.04878,0.011772,36,0.878049,0.861111,3,0.073171,0.097561,51,0,0.0,0.0,15,0.294118,0.257143,4,0.078431,0.117647,32,0.627451,0.606061,157.535338,false
"""J000/J000427.37-003932.1.png""","""J000427.37-003932.1""",1.114083,-0.658916,0.139308,-20.835133,83.305145,1.656927,5.593759,3.932198,"""pre_active""",false,false,"""DR1""",45,37,0.822222,0.311689,2,0.044444,0.65625,6,0.133333,0.162793,37,37,1.0,1.0,0,0.0,0.0,0,0.0,0.0,2,0,0.0,…,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,2,0,0.0,0.0,2,1.0,1.0,0,0.0,0.0,39,3,0.076923,0.079406,5,0.128205,0.21875,